# Data-X Spring 2019: Homework 7

### Webscraping



In this homework, you will do some exercises with web-scraping.

## Name: Junyan Tan

## SID: 3033370095



### Fun with Webscraping & Text manipulation


## 1. Statistics in Presidential Debates

Your first task is to scrape Presidential Debates from the Commission of Presidential Debates website: https://www.debates.org/voter-education/debate-transcripts/

To do this, you are not allowed to manually look up the URLs that you need, instead you have to scrape them. The root url to be scraped is the one listed above, namely: https://www.debates.org/voter-education/debate-transcripts/


1. By using `requests` and `BeautifulSoup` find all the links / URLs on the website that links to transcriptions of **First Presidential Debates** from the years [1988, 1984, 1976, 1960]. In total you should find 4 links / URLs that fulfill this criteria. **Print the urls.**

2. When you have a list of the URLs your task is to create a Data Frame with some statistics (see example of output below):
    1. Scrape the title of each link and use that as the column name in your Data Frame. 
    2. Count how long the transcript of the debate is (as in the number of characters in transcription string). Feel free to include `\` characters in your count, but remove any breakline characters, i.e. `\n`. You will get credit if your count is +/- 10% from our result.
    3. Count how many times the word **war** was used in the different debates. Note that you have to convert the text in a smart way (to not count the word **warranty** for example, but counting **war.**, **war!**, **war,** or **War** etc.
    4. Also scrape the most common used word in the debate, and write how many times it was used. Note that you have to use the same strategy as in C in order to do this.
    
    **Print your final output result.**
    
**Tips:**

___

In order to solve the questions above, it can be useful to work with Regular Expressions and explore methods on strings like `.strip(), .replace(), .find(), .count(), .lower()` etc. Both are very powerful tools to do string processing in Python. To count common words for example I used a `Counter` object and a Regular expression pattern for only words, see example:

```python
    from collections import Counter
    import re

    counts = Counter(re.findall(r"[\w']+", text.lower()))
```

Read more about Regular Expressions here: https://docs.python.org/3/howto/regex.html
    
    
**Example output of all of the answers to Question 1.2:**


![pres_stats_2](https://github.com/ikhlaqsidhu/data-x/raw/master/x-archive/misc/hw2_imgs_spring2018/presidents_stats_2.jpg)



----

.




In [1]:
# Import libraries
import requests
import bs4 as bs
import pandas as pd
from collections import Counter
import re
import numpy as np

In [2]:
# Get sources from the website
source = requests.get('https://www.debates.org/voter-education/debate-transcripts/').content
soup = bs.BeautifulSoup(source, features='html.parser')

In [4]:
"""
1. Find all the links / URLs on the website that links to transcriptions 
of First Presidential Debates from the years [1988, 1984, 1976, 1960].
2A. Scrape the title of each link and use that as the column name in your Data Frame
"""
links = []
ID = ['1988', '1984', '1976', '1960']
titles = []
for url in soup.find_all('a'):
    link = url.get('href')
    for i in ID:
        if i in link:
            if 'First' in url.text:
                links.append('https://www.debates.org' + link)
                titles.append(url.text)
display(links)
display(titles)

['https://www.debates.org/voter-education/debate-transcripts/september-25-1988-debate-transcript/',
 'https://www.debates.org/voter-education/debate-transcripts/october-7-1984-debate-transcript/',
 'https://www.debates.org/voter-education/debate-transcripts/september-23-1976-debate-transcript/',
 'https://www.debates.org/voter-education/debate-transcripts/september-26-1960-debate-transcript/']

['September 25, 1988: The First Bush-Dukakis Presidential Debate',
 'October 7, 1984: The First Reagan-Mondale Presidential Debate',
 'September 23, 1976: The First Carter-Ford Presidential Debate',
 'September 26, 1960: The First Kennedy-Nixon Presidential Debate']

In [8]:
"""
2B. Count how long the transcript of the debate is (as in the number of characters in transcription string). 
    Feel free to include \ characters in your count, but remove any breakline characters,
    i.e. \n. You will get credit if your count is +/- 10% from our result.
2C. Count how many times the word war was used in the different debates. Note that you have to 
    convert the text in a smart way 
   (to not count the word warranty for example, but counting war., war!, war, or War etc.
2D. Also scrape the most common used word in the debate, and write how many times it was used. 
    Note that you have to use the same strategy as in C in order to do this.
"""

num_chars = []
num_wars = []
num_words = []
for link in links:
    source_i = requests.get(link).content
    soup_i = bs.BeautifulSoup(source_i, features='html.parser')
    num_chars.append(len(re.sub(r"<.*?>", "", str(soup_i.find_all('p'))).replace("\n", "").replace("\r", "")
                         .replace("[", "").replace("]", "")))
    counts_war = Counter(re.findall(r"\bwar\b", str(soup_i.find_all('p')).lower()))
    num_wars.append(counts_war)
    counts_word = Counter(soup_i.text.lower().split()).most_common(1)
    num_words.append(counts_word)

display(num_chars)
display(num_wars)
display(num_words)

[88005, 87135, 80969, 61120]

[Counter({'war': 8}),
 Counter({'war': 2}),
 Counter({'war': 7}),
 Counter({'war': 3})]

[[('the', 800)], [('the', 868)], [('the', 857)], [('the', 780)]]

In [9]:
# Create a dataframe

debates = pd.DataFrame(columns=titles, index=["Debate char length", "war_count", 
                                              "most_common_w", "most_common_w_count"])
debates.loc["Debate char length"] = num_chars
debates.loc["war_count"] = [8, 2, 7, 3]
debates.loc["most_common_w"] = ["the", "the", "the", "the"]
debates.loc["most_common_w_count"] = [800, 868, 857, 780]
debates

,"September 25, 1988: The First Bush-Dukakis Presidential Debate","October 7, 1984: The First Reagan-Mondale Presidential Debate","September 23, 1976: The First Carter-Ford Presidential Debate","September 26, 1960: The First Kennedy-Nixon Presidential Debate"
Debate char length,88005,87135,80969,61120
war_count,8,2,7,3
most_common_w,the,the,the,the
most_common_w_count,800,868,857,780


    
## 2. Download and read in specific line from many data sets

Scrape the first 27 data sets from this URL http://people.sc.fsu.edu/~jburkardt/datasets/regression/ (i.e.`x01.txt` - `x27.txt`). Then, save the 5th line in each data set, this should be the name of the data set author (get rid of the `#` symbol, the white spaces and the comma at the end). 

Count how many times (with a Python function) each author is the reference for one of the 27 data sets. Showcase your results, sorted, with the most common author name first and how many times he appeared in data sets. Use a Pandas DataFrame to show your results, see example. **Print your final output result.**

**Example output of the answer for Question 2:**

![author_stats](https://github.com/ikhlaqsidhu/data-x/raw/master/x-archive/misc/hw2_imgs_spring2018/data_authors.png)


In [10]:
source2 = requests.get('http://people.sc.fsu.edu/~jburkardt/datasets/regression/').content
soup2 = bs.BeautifulSoup(source2, features='html.parser')

In [11]:
# Scrape the first 27 data sets

txts = []
for url in soup2.find_all('a'):
    txt = url.get('href')
    if 'txt' in txt:
        txts.append("http://people.sc.fsu.edu/~jburkardt/datasets/regression/" + txt)
txts = txts[0:27]
txts

['http://people.sc.fsu.edu/~jburkardt/datasets/regression/x01.txt',
 'http://people.sc.fsu.edu/~jburkardt/datasets/regression/x02.txt',
 'http://people.sc.fsu.edu/~jburkardt/datasets/regression/x03.txt',
 'http://people.sc.fsu.edu/~jburkardt/datasets/regression/x04.txt',
 'http://people.sc.fsu.edu/~jburkardt/datasets/regression/x05.txt',
 'http://people.sc.fsu.edu/~jburkardt/datasets/regression/x06.txt',
 'http://people.sc.fsu.edu/~jburkardt/datasets/regression/x07.txt',
 'http://people.sc.fsu.edu/~jburkardt/datasets/regression/x08.txt',
 'http://people.sc.fsu.edu/~jburkardt/datasets/regression/x09.txt',
 'http://people.sc.fsu.edu/~jburkardt/datasets/regression/x10.txt',
 'http://people.sc.fsu.edu/~jburkardt/datasets/regression/x11.txt',
 'http://people.sc.fsu.edu/~jburkardt/datasets/regression/x12.txt',
 'http://people.sc.fsu.edu/~jburkardt/datasets/regression/x13.txt',
 'http://people.sc.fsu.edu/~jburkardt/datasets/regression/x14.txt',
 'http://people.sc.fsu.edu/~jburkardt/datasets/r

In [12]:
"""
Save the 5th line in each data set, this should be the name of the data set author 
    (get rid of the # symbol, the white spaces and the comma at the end).
"""

names = []
for link in txts:
    source_n = requests.get(link).content
    soup_n = bs.BeautifulSoup(source_n, features='html.parser')
    names.append(re.sub(r"^#\s*", "", soup_n.text.split("\n")[4]).replace(",", ""))

In [13]:
names

['Helmut Spaeth',
 'Helmut Spaeth',
 'Helmut Spaeth',
 'Helmut Spaeth',
 'Helmut Spaeth',
 'R J Freund and P D Minton',
 'D G Kleinbaum and L L Kupper',
 'Helmut Spaeth',
 'D G Kleinbaum and L L Kupper',
 'K A Brownlee',
 'Helmut Spaeth',
 'Helmut Spaeth',
 'S Chatterjee and B Price',
 'Helmut Spaeth',
 'Helmut Spaeth',
 'Helmut Spaeth',
 'Helmut Spaeth',
 'Helmut Spaeth',
 'R J Freund and P D Minton',
 'Helmut Spaeth',
 'Helmut Spaeth',
 'Helmut Spaeth',
 'S Chatterjee B Price',
 'S Chatterjee B Price',
 'S Chatterjee B Price',
 'S C Narula J F Wellington',
 'S C Narula J F Wellington']

In [14]:
"""
Count how many times (with a Python function) each author is the reference for one of the 27 data sets. 
    Showcase your results, sorted, with the most common author name first and how many times he appeared in data sets.
    Create a panda dataframe for the result.
"""

names_count = Counter(names).most_common(7)
names_count

[('Helmut Spaeth', 16),
 ('S Chatterjee B Price', 3),
 ('R J Freund and P D Minton', 2),
 ('D G Kleinbaum and L L Kupper', 2),
 ('S C Narula J F Wellington', 2),
 ('K A Brownlee', 1),
 ('S Chatterjee and B Price', 1)]

In [25]:
author_tbl = pd.DataFrame(names_count, columns={"Authors", "Counts"})
author_tbl.loc[:, ['Authors','Counts']] = author_tbl.loc[:, ['Counts','Authors']].values
author_tbl.set_index("Authors")

,Counts
Authors,
Helmut Spaeth,16
S Chatterjee B Price,3
R J Freund and P D Minton,2
D G Kleinbaum and L L Kupper,2
S C Narula J F Wellington,2
K A Brownlee,1
S Chatterjee and B Price,1
